In [ ]:
!pip install praw

import praw
import pandas as pd

# Initialize Reddit API
reddit = praw.Reddit(client_id="0xSv0PGhlJdNwVSI5B2uDQ",
                     client_secret="JF5vcI4pryVsnibtZYvxz1171zn-aA",
                     user_agent="RVA 1.0")

# List of subreddits to analyze
subreddits = ["technology", "machinelearning", "datascience"]

# Initialize empty list to store posts
posts = []

# Loop through each subreddit
for sub in subreddits:
    subreddit = reddit.subreddit(sub)  # Access each subreddit
    for post in subreddit.top(limit=1000):  # Fetch top posts
        # Determine post type: text post or link post
        post_type = "link" if post.url else "text"

        # Append post data including subreddit, post type, and post length
        posts.append([
            post.title,
            post.score,
            post.num_comments,
            post.created_utc,
            sub,
            post_type,
            len(post.selftext)  # Post Length
        ])

# Convert the list to DataFrame
df = pd.DataFrame(posts, columns=[
    "title", "upvotes", "num_comments", "created_utc", "subreddit", "post_type", "post_length"
])

# Show the first few rows of the DataFrame
print(df.head())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.9 MB/s eta 0:00:00


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

                                               title  upvotes  num_comments  \
0  Hundreds of Subreddits Are Considering Banning...   171584          7064   
1        Reddit bans subreddit group "r/DonaldTrump"   147263         10288   
2  Congress has set out a bill to stop the FCC ta...   140035          1540   
3  John Oliver Blackmails Congress With Their Own...   133041          5102   
4  California-based game company Blizzard bans pr...   129872          6916   

    created_utc   subreddit post_type  post_length  
0  1.737568e+09  technology      link            0  
1  1.610127e+09  technology      link            0  
2  1.513037e+09  technology      link            0  
3  1.649682e+09  technology      link            0  
4  1.570548e+09  technology      link            0  


In [ ]:
df.size

20881

In [ ]:
# Assuming df is your DataFrame
df.to_csv('Redit_data.csv', index=False)


In [ ]:
df.head()

,title,upvotes,num_comments,created_utc,subreddit,post_type,post_length
0,Hundreds of Subreddits Are Considering Banning...,171584,7064,1.737568e+09,technology,link,0
1,"Reddit bans subreddit group ""r/DonaldTrump""",147263,10288,1.610127e+09,technology,link,0
2,Congress has set out a bill to stop the FCC ta...,140035,1540,1.513037e+09,technology,link,0
3,John Oliver Blackmails Congress With Their Own...,133041,5102,1.649682e+09,technology,link,0
4,California-based game company Blizzard bans pr...,129872,6916,1.570548e+09,technology,link,0


In [ ]:
threshold = df["upvotes"].quantile(0.80)  # Top 20% upvotes
df["viral"] = (df["upvotes"] >= threshold).astype(int)


In [ ]:
import datetime

df["created_utc"] = pd.to_datetime(df["created_utc"], unit="s")
df["hour"] = df["created_utc"].dt.hour  # Posting hour
df["day_of_week"] = df["created_utc"].dt.dayofweek  # 0=Monday, 6=Sunday


In [ ]:
from textblob import TextBlob

df["title_length"] = df["title"].apply(lambda x: len(x.split()))
df["sentiment"] = df["title"].apply(lambda x: TextBlob(x).sentiment.polarity)


In [ ]:
from sklearn.model_selection import train_test_split

# Select features
features = ["title_length", "num_comments", "hour", "day_of_week", "sentiment"]
X = df[features]
y = df["viral"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Traning for lofistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8324958123953099
              precision    recall  f1-score   support

           0       0.85      0.95      0.90       459
           1       0.73      0.43      0.55       138

    accuracy                           0.83       597
   macro avg       0.79      0.69      0.72       597
weighted avg       0.82      0.83      0.82       597



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Traning random forest

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
print("Random Forest Accuracy:", rf.score(X_test, y_test))


Random Forest Accuracy: 0.8358458961474037


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train, y_train)
print("XGBoost Accuracy:", xgb.score(X_test, y_test))


XGBoost Accuracy: 0.8224455611390284


In [ ]:
#Hyper parameter tuning

from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 10]}
grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=3)
grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)


Best Parameters: {'max_depth': 5, 'n_estimators': 50}


In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix

print("AUC-ROC:", roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


AUC-ROC: 0.9178822897919232
Confusion Matrix:
 [[437  22]
 [ 78  60]]


In [ ]:
import pickle

# Save the model
with open('redit.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load the model back
with open('redit.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
